<a href="https://colab.research.google.com/github/samkrem/ACL_UWB_SLAM/blob/main/UWB_Sensor_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import necessary libraries and files
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from pathlib import Path




In [ ]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
#posefileName="swarm/workspaces/UWB/atb.bin"

def binFileToStr(file_path):
  with open(file_path, "rb") as file:
    file_str = file.read()
  chosen_encoding = 'ascii'
  try:
    decoded_file_str = binary_data.decode(chosen_encoding)
  except UnicodeDecodeError as e:
    print(f"Error decoding binary data: {e}")
  return decoded_file_str

UWB_file_path="atb.bin"
UWB_pose=binFileToStr(UWB_file_path)

true_pose_file_path= "atb.bin"
true_pose=binFileToStr(true_pose_file_path)



In [ ]:
def stringToTensorGPU (pose_str):
  #first convert to list then to numpy array

  pose_str=pose_str.replace("\n", "")
  pose_np_unshaped = np.fromstring(pose_str, dtype= float, sep=' ')
  pose_np=np.reshape(pose_np_unshaped,(-1, 6))
  pose_tensor_GPU=torch.tensor(pose_np, dtype=torch.float32).to(device)

  return pose_tensor_GPU

X_ultrawideband=stringToTensorGPU(UWB_pose)
y_true_pose=stringToTensorGPU(true_pose)

#for just train test
# X_train, X_test, y_train, y_test = train_test_split(X_ultrawideband, y_true_pose, test_size=0.3, random_state=42)

# for train, validation, test
# X_train, X_validtest, y_train, y_validtest = train_test_split(X_ultrawideband, y_true_pose, test_size=0.3, random_state=42)
# X_valid, X_test, y_valid, y_test= train_test_split(X_validtest, y_validtest, test_size=0.5, random_state=42)


In [ ]:
uwb_seed=42
np.random.seed(uwb_seed)
X_ultrawideband = np.random.rand(1000, 6)  # 1000 samples, 6 features (x, y, z, roll, pitch, yaw)

true_pose_seed=43
np.random.seed(true_pose_seed)
y_true_pose = np.random.rand(1000, 6)       # Corresponding true poses # think ab out if true pose is X vs y
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_ultrawideband, y_true_pose, test_size=0.2, random_state=42)


In [ ]:
# Standardize the input data
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train)
#X_valid_scaled= X_scaler.fit_transform(X_valid)
X_test_scaled = X_scaler.fit_transform(X_test)

#Standardize the true data
y_scaler= StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
#y_valid_scaled= y_scaler.fit_transform(y_valid)

y_test_scaled = y_scaler.fit_transform(y_test)


In [ ]:
# Convert data to PyTorch tensors then convert to GPU THIS IS UNESCESARY WITH THE GPU METHOD because we already have tensors in GPU
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

In [ ]:
# Define the neural network model
class BiasCorrectionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6, 64) #trial and error, adjust/look at hyperparameters
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 6)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
model= BiasCorrectionModel()
model.to(device)
#print(f"model.state_dict(): {model.state_dict()}")

fc1weight_shape=model.state_dict()["fc1.weight"].shape
fc1bias_shape=model.state_dict()["fc1.bias"].shape

fc2weight_shape=model.state_dict()["fc2.weight"].shape
fc2bias_shape=model.state_dict()["fc2.bias"].shape

fc3weight_shape=model.state_dict()["fc3.weight"].shape
fc3bias_shape=model.state_dict()["fc3.bias"].shape

print(f"model state dict keys: {model.state_dict().keys()}")
print(f"fc1weight_shape: {fc1weight_shape} | fc1bias_shape: {fc1bias_shape}")
print(f"fc2weight_shape: {fc2weight_shape} | fc2bias_shape: {fc2bias_shape}")
print(f"fc3weight_shape: {fc3weight_shape} | fc3bias_shape: {fc3bias_shape}")


model state dict keys: odict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
fc1weight_shape: torch.Size([64, 6]) | fc1bias_shape: torch.Size([64])
fc2weight_shape: torch.Size([64, 64]) | fc2bias_shape: torch.Size([64])
fc3weight_shape: torch.Size([6, 64]) | fc3bias_shape: torch.Size([6])


In [ ]:
#Setup loss function & optimizer
loss_fn = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
#Set up batches

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [ ]:
epochs=500
batch_size=64

epoch_count= []
loss_values= []
test_loss_values = []


for epoch in range(epochs):
  #Set model to training mode
  model.train()
  for batch_X, batch_y in train_loader:
    #1. Forward pass
    y_batch_pred = model(batch_X) #predicted pose essentially? or is this the error

    #2. Calculate loss
    loss = loss_fn(y_batch_pred, batch_y)    #comparing loss between predictions and real pose data

    #3. Zero the gradient
    optimizer.zero_grad()

    #3. Perform backpropogation:
    loss.backward()

    #4. Step the optimizer
    optimizer.step()

  model.eval()
  with torch.inference_mode(): #testing the predictions
    test_pred=model(X_test_tensor)
    test_loss=loss_fn(test_pred, y_test_tensor)

  if epoch%100==0: #just to print info
    epoch_count.append(epoch)
    loss_values.append(loss)
    test_loss_values.append(test_loss)
    print(f"Epoch:{epoch}| Loss: {loss} | Test loss: {test_loss}")


model.eval()
with torch.inference_mode():
    std_corrected_UWB_pose = model(torch.tensor(X_ultrawideband, dtype=torch.float32).to(device))
    std_corrected_UWB_pose_np= std_corrected_UWB_pose.cpu().numpy()




Epoch:0| Loss: 0.023785963654518127 | Test loss: 0.16352808475494385
Epoch:100| Loss: 0.02378189191222191 | Test loss: 0.1671537607908249
Epoch:200| Loss: 0.020764712244272232 | Test loss: 0.1675221025943756
Epoch:300| Loss: 0.02178901433944702 | Test loss: 0.17068596184253693
Epoch:400| Loss: 0.02159104496240616 | Test loss: 0.17230242490768433


In [ ]:
    unstd_pose_pred=X_scaler.inverse_transform(std_corrected_UWB_pose_np) #undstandardizes the pose


In [ ]:
print(std_corrected_UWB_pose_np)

print(unstd_pose_pred)

print(y_test_tensor)

[[0.728162   0.28526983 0.24886122 0.64477015 0.77886206 0.54700196]
 [0.6983419  0.27485135 0.20278019 0.8154452  1.1004988  0.7042091 ]
 [0.678759   0.43025148 0.6197879  0.92913306 1.2434137  0.3930892 ]
 ...
 [0.9425957  0.6889063  0.6899925  0.9854301  0.6846244  0.41858158]
 [0.5449331  0.10044131 0.0951969  0.9601774  0.86527735 0.84883255]
 [0.8296205  0.5704891  0.4072615  0.6173371  0.52849865 0.56471837]]
[[0.70613974 0.56530404 0.5483534  0.68999493 0.7208014  0.6731396 ]
 [0.69739294 0.5623857  0.5342208  0.7365346  0.81533647 0.721369  ]
 [0.69164896 0.6059157  0.66211283 0.76753503 0.8573418  0.62592083]
 ...
 [0.76903707 0.6783691  0.6836439  0.7828861  0.6931032  0.6337416 ]
 [0.65239537 0.5135306  0.50122607 0.7760002  0.7462005  0.76573795]
 [0.7358994  0.6451986  0.5969331  0.6825145  0.64721495 0.6785748 ]]
tensor([[0.7750, 0.0772, 0.4166, 0.5351, 0.3047, 0.2069],
        [0.5821, 0.4093, 0.5621, 0.9074, 0.3667, 0.2544],
        [0.4084, 0.3484, 0.6304, 0.9612, 0.6

tensor([[0.7750, 0.0772, 0.4166, 0.5351, 0.3047, 0.2069],
        [0.5821, 0.4093, 0.5621, 0.9074, 0.3667, 0.2544],
        [0.4084, 0.3484, 0.6304, 0.9612, 0.6997, 0.8898],
        ...,
        [0.5273, 0.8679, 0.1368, 0.8230, 0.4618, 0.2125],
        [0.7165, 0.1977, 0.5968, 0.9926, 0.2750, 0.4953],
        [0.6008, 0.8183, 0.2408, 0.5582, 0.1723, 0.7926]], device='cuda:0')


In [ ]:
#Input value X
print(f"y_true_pose[0][0]: {y_true_pose[0][0]}")
print(f"unstd_pose_pred[0][0]: {unstd_pose_pred[0][0]}")

y_true_pose[0][0]: 0.11505456638977896
unstd_pose_pred[0][0]: 0.7061397433280945


In [ ]:
#Saving model

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME="UWB_Sensor_Correction_Model.pth" #pytorch saved as .pth /.pt
MODEL_SAVE_PATH= MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)


# Create new instance of model and load saved state dict (make sure to put it on the target device)
loaded_model=BiasCorrectionModel()
loaded_model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))
loaded_model.to(device)
next(loaded_model.parameters()).device



loaded_model.to(device)
next(loaded_model.parameters()).device